<a href="https://colab.research.google.com/github/LeeSeungwon89/Lecture-and-self-study/blob/master/5-1%20%EA%B2%B0%EC%A0%95%20%ED%8A%B8%EB%A6%AC(%EC%A0%95%EB%A6%AC%20%EC%A4%91).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 결정 트리

## 로지스틱 회귀로 와인 분류하기

### 데이터 준비

In [11]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')
print(wine)
# 총 샘플 6496개로 이루어진 판다스 데이터프레임이며,
# 네 번째 열은 타깃값으로 0(음성 클래스)이면 레드 와인, 1(양성 클래스)이면 화이트 와인임.
# 즉 화이트 와인을 골라내는 문제임.

      alcohol  sugar    pH  class
0         9.4    1.9  3.51    0.0
1         9.8    2.6  3.20    0.0
2         9.8    2.3  3.26    0.0
3         9.8    1.9  3.16    0.0
4         9.4    1.9  3.51    0.0
...       ...    ...   ...    ...
6492     11.2    1.6  3.27    1.0
6493      9.6    8.0  3.15    1.0
6494      9.4    1.2  2.99    1.0
6495     12.8    1.1  3.34    1.0
6496     11.8    0.8  3.26    1.0

[6497 rows x 4 columns]


In [12]:
# info() 메서드를 사용하여 누락된 데이터가 있는지 확인함.
wine.info()
# 누락된 값은 없음.
# 만약 누락된 값이 있다면 그 데이터를 버리거나 평균값으로 채우고 사용함.
# 최선의 방법이 정해진 건 아니므로 두 방법 모두 사용해보고 최선을 택함..

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB


In [13]:
# describe() 메서드를 사용하여 열에 대한 간략한 통계를 출력함.
wine.describe()
# 개수, 평균, 표준편차, 최소, 최대, 1사분위수 ~ 최대까지 알려줌.
# 만약 데이터 개수가 짝수개라서 중앙값을 선택하기 어려운 경우는 가운데 값 2개의 평균을 사용함.
# 각 특성의 스케일이 다르므로 StandardScaler 클래스를 사용하여 특성을 표준화 전처리 해야 함.

,alcohol,sugar,pH,class
count,6497.000000,6497.000000,6497.000000,6497.000000
mean,10.491801,5.443235,3.218501,0.753886
std,1.192712,4.757804,0.160787,0.430779
min,8.000000,0.600000,2.720000,0.000000
25%,9.500000,1.800000,3.110000,1.000000
50%,10.300000,3.000000,3.210000,1.000000
75%,11.300000,8.100000,3.320000,1.000000
max,14.900000,65.800000,4.010000,1.000000


In [19]:
# 판다스 데이터프레임을 넘파이 배열로 바꾸고, 훈련 세트와 테스트 세트로 나눔.
data = wine[['alcohol', 'sugar', 'pH']].to_numpy() # 특성만 골라서 2차원 리스트로 만듦.
print(data)

target = wine['class'].to_numpy() # 타깃만 골라서 1차원 리스트로 만듦.
print(target)

from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)
# test_size 매개변수의 기본값은 0.25(25%). 샘플 개수가 충분히 많으므로 0.2로 지정함.

print(train_input.shape, test_input.shape)

[[ 9.4   1.9   3.51]
 [ 9.8   2.6   3.2 ]
 [ 9.8   2.3   3.26]
 ...
 [ 9.4   1.2   2.99]
 [12.8   1.1   3.34]
 [11.8   0.8   3.26]]
[0. 0. 0. ... 1. 1. 1.]
(5197, 3) (1300, 3)


In [23]:
# StandardScaler 클래스를 사용하여 훈련 세트를 전처리 함.
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

ss.fit(train_input)

train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

print(train_scaled, test_scaled)

[[ 0.01376574  0.4646116  -0.16888369]
 [ 1.60871671 -0.70299854  0.26967864]
 [ 1.10504798 -0.70299854  1.20945505]
 ...
 [-0.82568214  0.0267578   0.14437512]
 [-0.82568214  0.52716215 -0.23153545]
 [-0.82568214 -0.57789745  1.83597265]] [[ 1.44082714  1.52797084  0.26967864]
 [-0.48990299 -0.68214836  0.3323304 ]
 [ 1.27293756  0.40206106 -0.23153545]
 ...
 [ 1.60871671 -0.76554908 -0.16888369]
 [-0.90962693  0.88161523  0.52028568]
 [-1.49724045  2.09092573 -1.98578474]]


In [25]:
# 로지스틱 회귀 모델을 훈련함.
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(train_scaled, train_target)

print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))
# 과소적합 상태임. 

0.7808350971714451
0.7776923076923077
